In [1]:
import os
os.chdir('tsp_dataset')

# Parse input 

In [43]:
def parse(fileaddress):
    CoOrdinates = {}
    with open(fileaddress) as f:
        lines = f.readlines()
    Name = lines[0].split(" ")[1].strip()
    EWT = lines[4].split(" ")[1].strip()
    Dimension = lines[3].split(" ")[1].strip()
    lines = lines[6:-2]
    for line in lines:
        n,x,y = line.strip().split(" ")
        CoOrdinates[n] = (x,y)
    return Name, EWT, Dimension, CoOrdinates

<p>we save the coordinates in a dictionary with the vertex ID as the key and a (x,y) tuple as the data</p>

In [46]:
NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates  = parse('berlin52.tsp')